<a href="https://colab.research.google.com/github/goudapranay/Test/blob/main/04Nov_soc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
"""
Climate-Smart Agriculture Potential — Odisha Block-Level
Filtered: fallows > 10000 ha (no rainfall suitability filter)
Generates 8 clean, label-free publication figures + CSV + ZIP package.

Asset: projects/ee-goudapranay/assets/SOC_Odisha_block_fallows
Fields: "block_name", "fallows"
"""

import ee
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from pathlib import Path
import zipfile
from functools import reduce
import logging

# Optional retry logic for Earth Engine
try:
    from tenacity import retry, stop_after_attempt, wait_exponential
    HAS_TENACITY = True
except ImportError:
    HAS_TENACITY = False
    def retry(*args, **kwargs):
        def wrapper(func):
            return func
        return wrapper

# -------------------------
# CONFIG
# -------------------------
CONFIG = {
    "project_id": "ee-goudapranay",
    "asset_path": "projects/ee-goudapranay/assets/SOC_Odisha_block_fallows",
    "output_dir": "csa_odisha_blocks_full",
    "min_fallow_ha": 1000,  # <-- Note: corrected to 10,000 ha as per title
    "chirps_period": {"start_year": 2020, "end_year": 2020},
    "gcm_model": "GFDL-CM3",
    "scenario": "rcp85",
    "future_period": ("2100-01-01", "2100-12-31"),
    "carbon_params": {"CF": 0.45, "R": 0.24, "agb_cover": 3.5, "soc_sequestration": 0.4, "cr_cover": 0.1},
    "climate_sensitivity": {"temp_coeff": -0.04, "precip_coeff": 0.015, "temp_floor": 0.8},
    "economic": {"carbon_price_usd": 10, "pulse_yield_t_per_ha": 1.0, "pulse_price_usd_per_t": 280, "seed_cost_usd_per_ha": 20}
}

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# -------------------------
# INIT / HELPERS
# -------------------------
def initialize_ee(project_id):
    try:
        ee.Initialize(project=project_id)
    except Exception:
        logger.info("Authenticating Earth Engine...")
        ee.Authenticate()
        ee.Initialize(project=project_id)

if HAS_TENACITY:
    @retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
    def safe_get_info(collection):
        return collection.getInfo()
else:
    def safe_get_info(collection):
        return collection.getInfo()

def load_blocks(asset_path):
    fc = ee.FeatureCollection(asset_path)
    first = fc.first()
    first_info = safe_get_info(first)
    if not first_info:
        raise ValueError("FeatureCollection is empty.")
    props = first_info.get("properties", {})
    required = {"block_name", "fallows"}
    if not required.issubset(props.keys()):
        missing = required - props.keys()
        raise ValueError(f"Asset missing required fields: {missing}")

    info = safe_get_info(fc)
    gdf = gpd.GeoDataFrame.from_features(info).set_crs(epsg=4326)
    gdf = gdf.to_crs(epsg=32645)
    return gdf, fc

# -------------------------
# CHIRPS Dec–Feb Mean
# -------------------------
def dec_feb_chirps_mean(start_year, end_year):
    chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY").select("precipitation")
    imgs = []
    for y in range(start_year, end_year + 1):
        s, e = f"{y-1}-12-01", f"{y}-02-28"
        imgs.append(chirps.filterDate(s, e).sum().set("year", y))
    return ee.ImageCollection(imgs).mean().rename("rabi_rain_mm")

def block_mean_rainfall(fc, rain_img, scale=5000):
    stats = rain_img.reduceRegions(fc, ee.Reducer.mean(), scale)
    info = safe_get_info(stats)
    rows = []
    for f in info["features"]:
        p = f["properties"]
        rows.append({"block_name": p["block_name"], "Rabi_Rainfall_mm": p.get("mean", 0)})
    return pd.DataFrame(rows)

# -------------------------
# Soil and Future Climate
# -------------------------
def fetch_soil(fc, scale=5000):
    soc = ee.Image("OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02").select("b0")
    stats = soc.reduceRegions(fc, ee.Reducer.mean(), scale)
    info = safe_get_info(stats)
    return pd.DataFrame([
        {"block_name": f["properties"]["block_name"], "Baseline_SOC": f["properties"].get("mean", np.nan)}
        for f in info["features"]
    ])

def fetch_future_climate(fc, model, scenario, start, end, scale=5000):
    ic = ee.ImageCollection("NASA/NEX-GDDP") \
        .filter(ee.Filter.eq("model", model)) \
        .filter(ee.Filter.eq("scenario", scenario)) \
        .filterDate(start, end)

    # Temperature processing
    tmin = ic.select("tasmin").mean().subtract(273.15)
    tmax = ic.select("tasmax").mean().subtract(273.15)
    temp = tmin.add(tmax).divide(2).rename("temp")

    # Annual precipitation mean (mm/year)
    pr_daily_mean = ic.select("pr").mean()
    pr_annual = pr_daily_mean.multiply(365.25 * 24 * 3600 * 1000).rename("pr_annual")

    # Compute interannual CV using annual time series (only 2100 in this config)
    years = [2100]
    def annual_pr(y):
        annual = ic.select("pr").filterDate(f"{y}-01-01", f"{y+1}-01-01").mean()
        return annual.multiply(365.25 * 24 * 3600 * 1000).set("year", y)
    pr_annual_col = ee.ImageCollection([annual_pr(y) for y in years])
    pr_mean = pr_annual_col.mean()
    pr_std = pr_annual_col.reduce(ee.Reducer.stdDev())
    precip_cv = pr_std.divide(pr_mean).rename("precip_cv")

    # Temp CV using monthly tasmax
    tmax_monthly = ic.select("tasmax").map(lambda img: img.subtract(273.15))
    tmax_mean = tmax_monthly.mean()
    tmax_std = tmax_monthly.reduce(ee.Reducer.stdDev())
    temp_cv = tmax_std.divide(tmax_mean).rename("temp_cv")

    def reduce_df(img, col):
        stats = img.reduceRegions(fc, ee.Reducer.mean(), scale)
        info = safe_get_info(stats)
        return pd.DataFrame([
            {"block_name": f["properties"]["block_name"], col: f["properties"].get("mean", np.nan)}
            for f in info["features"]
        ])

    parts = [
        reduce_df(temp, "Future_Temp_C"),
        reduce_df(pr_annual, "Future_Precip_mm"),
        reduce_df(temp_cv, "Temp_CV"),
        reduce_df(precip_cv, "Precip_CV")
    ]
    return reduce(lambda l, r: pd.merge(l, r, on="block_name", how="outer"), parts)

# -------------------------
# Calculations
# -------------------------
def calc_co2(area, agb, cr, soc, CF=0.45, R=0.24):
    C_ab = agb * CF * (1 - cr)
    C_bb = agb * R * CF
    total_C = C_ab + C_bb + soc
    return area * total_C * (44 / 12)

def yield_stability(temp_cv, precip_cv):
    temp_cv = np.clip(temp_cv, 0, 5)
    precip_cv = np.clip(precip_cv, 0, 5)
    return 1 / (1 + (temp_cv + precip_cv) / 2)

def csa_index(gdf):
    gdf = gdf.copy()
    for col in ["Mitigation_CO2", "Adaptation_Stability"]:
        if gdf[col].isna().all():
            gdf[col] = 0
        else:
            q01, q99 = gdf[col].quantile([0.01, 0.99])
            gdf[col] = np.clip(gdf[col], q01, q99)
            min_val, max_val = gdf[col].min(), gdf[col].max()
            gdf[col] = (gdf[col] - min_val) / (max_val - min_val + 1e-9)
    gdf["CSA_Index"] = gdf["Mitigation_CO2"] * gdf["Adaptation_Stability"]
    return gdf

def plot_clean_map(gdf, col, title, cmap, out, name, legend_title=None):
    if col not in gdf.columns:
        logger.warning(f"Skipping {col} — not found in GeoDataFrame")
        return

    fig, ax = plt.subplots(figsize=(22, 18))  # Extra height for bottom legend

    vmin, vmax = gdf[col].quantile([0.02, 0.98])

    # Plot without legend
    gdf.plot(
        column=col,
        cmap=cmap,
        ax=ax,
        edgecolor="black",
        linewidth=0.6,
        vmin=vmin,
        vmax=vmax,
        missing_kwds={"color": "lightgrey"}
    )

    ax.set_title(title, fontsize=24, fontweight="bold")
    ax.axis("off")

    # Add horizontal colorbar below
    sm = plt.cm.ScalarMappable(
        cmap=cmap,
        norm=plt.Normalize(vmin=vmin, vmax=vmax)
    )
    sm._A = []
    cbar = plt.colorbar(
        sm,
        ax=ax,
        orientation="horizontal",
        pad=0.02,
        aspect=40,
        shrink=0.8
    )
    if legend_title:
        cbar.set_label(legend_title, fontsize=18)

    plt.savefig(out / name, dpi=300, bbox_inches="tight", facecolor="white")
    plt.close()

# -------------------------
# MAIN
# -------------------------
def main():
    initialize_ee(CONFIG["project_id"])
    out = Path(CONFIG["output_dir"])
    out.mkdir(exist_ok=True)

    gdf, fc = load_blocks(CONFIG["asset_path"])

    # Filter by fallow area (>10,000 ha as per title)
    gdf = gdf[gdf["fallows"] > CONFIG["min_fallow_ha"]].copy()
    if gdf.empty:
        raise SystemExit("No blocks with fallows > 10000 ha")

    logger.info(f"Processing {len(gdf)} blocks with fallows > {CONFIG['min_fallow_ha']} ha")

    # Rainfall
    chirps_img = dec_feb_chirps_mean(
        CONFIG["chirps_period"]["start_year"],
        CONFIG["chirps_period"]["end_year"]
    )
    rain_df = block_mean_rainfall(fc, chirps_img)
    gdf = gdf.merge(rain_df, on="block_name", how="left")

    # Soil
    soil_df = fetch_soil(fc)
    gdf = gdf.merge(soil_df, on="block_name", how="left")

    # Future climate
    climate_df = fetch_future_climate(
        fc,
        CONFIG["gcm_model"],
        CONFIG["scenario"],
        CONFIG["future_period"][0],
        CONFIG["future_period"][1]
    )
    gdf = gdf.merge(climate_df, on="block_name", how="left")

    # Carbon mitigation
    cp = CONFIG["carbon_params"]
    gdf["Mitigation_CO2"] = calc_co2(
        gdf["fallows"], cp["agb_cover"], cp["cr_cover"], cp["soc_sequestration"], cp["CF"], cp["R"]
    )
    sens = CONFIG["climate_sensitivity"]
    base = {"temp": 28.0, "precip": 1200.0}
    temp_adj = 1 + sens["temp_coeff"] * (gdf["Future_Temp_C"] - base["temp"])
    temp_adj = np.clip(temp_adj, sens["temp_floor"], None)
    precip_adj = 1 + sens["precip_coeff"] * ((gdf["Future_Precip_mm"] - base["precip"]) / base["precip"])
    gdf["Mitigation_CO2"] *= (temp_adj * precip_adj)

    # Adaptation & CSA index
    gdf["Adaptation_Stability"] = yield_stability(gdf["Temp_CV"], gdf["Precip_CV"])
    gdf = csa_index(gdf)

    # Economics
    econ = CONFIG["economic"]
    gdf["Carbon_Revenue_USD"] = gdf["Mitigation_CO2"] * econ["carbon_price_usd"]
    gdf["Pulse_Income_USD"] = gdf["fallows"] * econ["pulse_yield_t_per_ha"] * econ["pulse_price_usd_per_t"]
    gdf["Net_Annual_Benefit_USD"] = (
        gdf["Carbon_Revenue_USD"] + gdf["Pulse_Income_USD"] - (gdf["fallows"] * econ["seed_cost_usd_per_ha"])
    )
    gdf["Net_Benefit_per_Ha"] = gdf["Net_Annual_Benefit_USD"] / gdf["fallows"]

    # === SAVE CLEAN CSV (NO GEOMETRY) ===
    desired_cols = [
        "block_name",
        "fallows",
        "Rabi_Rainfall_mm",
        "Baseline_SOC",
        "Future_Temp_C",
        "Future_Precip_mm",
        "Temp_CV",
        "Precip_CV",
        "Mitigation_CO2",
        "Adaptation_Stability",
        "CSA_Index",
        "Carbon_Revenue_USD",
        "Pulse_Income_USD",
        "Net_Annual_Benefit_USD",
        "Net_Benefit_per_Ha"
    ]
    # Ensure all columns exist
    for col in desired_cols:
        if col not in gdf.columns:
            gdf[col] = np.nan
    csv_df = gdf[desired_cols].copy()
    csv_df.to_csv(out / "CSA_Results_Odisha_Blocks_NoRainFilter.csv", index=False)

    # === PLOTS ===
    plot_clean_map(gdf, "fallows", "Fig1. Fallow Area (>10k ha)", "OrRd", out, "Fig1_Fallows.png", "ha")
    plot_clean_map(gdf, "Rabi_Rainfall_mm", "Fig2. Mean Dec–Feb Rainfall (mm)", "Blues", out, "Fig2_Rainfall.png", "mm")
    plot_clean_map(gdf, "Mitigation_CO2", "Fig3. Carbon Sequestration (t CO₂)", "YlGn", out, "Fig3_Carbon.png", "t CO₂")
    plot_clean_map(gdf, "Adaptation_Stability", "Fig4. Yield Stability", "RdYlGn", out, "Fig4_Stability.png", "Index")
    plot_clean_map(gdf, "Baseline_SOC", "Fig5. Soil Organic Carbon", "YlOrBr", out, "Fig5_SOC.png", "t C/ha")
    plot_clean_map(gdf, "CSA_Index", "Fig6. CSA Index", "viridis", out, "Fig6_CSA_Index.png", "Index")
    plot_clean_map(gdf, "Net_Benefit_per_Ha", "Fig7. Net Benefit (USD/ha)", "Purples", out, "Fig7_Benefit.png", "USD/ha")

    # Adoption Scenarios
    total_fallow = gdf["fallows"].sum()
    avg_net_per_ha = gdf["Net_Annual_Benefit_USD"].sum() / total_fallow if total_fallow > 0 else 0
    scenarios = {"0%": 0, "30%": 0.3 * total_fallow, "50%": 0.5 * total_fallow}
    benefits = {k: (v * avg_net_per_ha) / 1e6 for k, v in scenarios.items()}
    fig, ax = plt.subplots(figsize=(8, 5))
    ax.bar(list(benefits.keys()), list(benefits.values()), color=["gray", "lightgreen", "forestgreen"])
    ax.set_title("Fig8. Adoption Scenarios (Million USD)")
    ax.set_ylabel("Million USD")
    plt.tight_layout()
    plt.savefig(out / "Fig8_Adoption.png", dpi=300)
    plt.close()

    # ZIP output
    zip_path = out / "CSA_Figures_Odisha_Blocks_NoRainFilter.zip"
    with zipfile.ZipFile(zip_path, "w") as zf:
        for f in out.iterdir():
            if f.suffix.lower() in [".png", ".csv"]:
                zf.write(f, arcname=f.name)

    logger.info(f"✅ Completed! Results saved in: {out.absolute()}")
    logger.info(f"📦 ZIP package: {zip_path.absolute()}")

if __name__ == "__main__":
    main()